<a href="https://colab.research.google.com/github/AjeetSingh02/Notebooks/blob/master/textClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset

In [1]:
#Loading the data set - training data.
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# You can check the target names (categories) and some data files by following commands.
twenty_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [4]:
atheism_christian_data = []
atheism_christian_target = []
atheism_christian_target_names = []

In [5]:
# Taking only two groups currently
for i in range(len(twenty_train.target)):
    if twenty_train.target[i] == 0 or twenty_train.target[i] == 15:
        atheism_christian_data.append(twenty_train.data[i])
        atheism_christian_target.append(twenty_train.target[i])
        atheism_christian_target_names.append(twenty_train.target_names[twenty_train.target[i]])

In [6]:
i = 0
print(atheism_christian_data[i])
print("Target Name: ", twenty_train.target_names[atheism_christian_target[i]])

From: mathew <mathew@mantis.co.uk>
Subject: Re: <Political Atheists?
Organization: Mantis Consultants, Cambridge. UK.
X-Newsreader: rusnews v1.01
Lines: 22

kmr4@po.CWRU.edu (Keith M. Ryan) writes:
> ( I am almost sure that Zyklon-B is immediate and painless method of 
> death. If not, insert soem other form. )
> 
>         And, ethnic and minority groups have been killed, mutilated and 
> exterminated through out history, so I guess it was not unusual.
> 
>         So, you would agree that the holocost would be allowed under the US 
> Constitution?  [ in so far, the punishment. I doubt they recieved what would 
> be considered a "fair" trial by US standards.

Don't be so sure.  Look what happened to Japanese citizens in the US during
World War II.  If you're prepared to say "Let's round these people up and
stick them in a concentration camp without trial", it's only a short step to
gassing them without trial.  After all, it seems that the Nazis originally
only intended to imprison the

In [7]:
i = 2
print(atheism_christian_data[i])
print("Target Name: ", twenty_train.target_names[atheism_christian_target[i]])

From: jonh@david.wheaton.edu (Jonathan Hayward)
Subject: Re: Pantheism & Environmentalism
Organization: Wheaton College, IL
Lines: 46

In article <Apr.5.23.31.36.1993.23919@athos.rutgers.edu> by028@cleveland.freenet.edu (Gary V. Cavano) writes:
>I'm new to this group, and maybe this has been covered already,
>but does anybody out there see the current emphasis on the
>environment being turned (unintentionally, of course) into
>pantheism?

Yes.

(I am adamantly an environmentalist.  I will not use styrofoam table service.
Please keep that in mind as you read this post - I do not wish to attack
environmentalism)

A half truth is at least as dangerous as a complete lie.  A complete lie will
rarely be readily accepted, while a half truth (the lie subtly hidden) is more
powerfully offered by one who masquerades as an angel of light.

Satan has (for some people) loosened the grip on treating the earth as something
other than God's intricate handiwork, something other than that on which the
h

# Data Preprocessing

In [8]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
import os
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
df = pd.DataFrame({"article": atheism_christian_data, 
                    "target_code": atheism_christian_target,
                    "taget_name": atheism_christian_target_names})

In [11]:
df["target_code"].where(df["target_code"] == 0, 1, inplace=True)
df.head()

,article,target_code,taget_name
0,From: mathew <mathew@mantis.co.uk>\nSubject: R...,0,alt.atheism
1,From: keith@cco.caltech.edu (Keith Allan Schne...,0,alt.atheism
2,From: jonh@david.wheaton.edu (Jonathan Hayward...,1,soc.religion.christian
3,From: dlecoint@garnet.acns.fsu.edu (Darius_Lec...,1,soc.religion.christian
4,From: a137490@lehtori.cc.tut.fi (Aario Sami)\n...,0,alt.atheism


In [12]:
df.target_code.value_counts()

1    599
0    480
Name: target_code, dtype: int64

In [13]:
# Tokenization
def tokenize_data(dataset):
    tokenizer = nltk.tokenize.TreebankWordTokenizer()
    for i in range(dataset.shape[0]):
       dataset["article"][i] = tokenizer.tokenize(dataset["article"][i])
       
    return dataset

In [14]:
df = tokenize_data(df)
print(df.head())
print(df["article"][0])

                                             article  ...              taget_name
0  [From, :, mathew, <, mathew, @, mantis.co.uk, ...  ...             alt.atheism
1  [From, :, keith, @, cco.caltech.edu, (, Keith,...  ...             alt.atheism
2  [From, :, jonh, @, david.wheaton.edu, (, Jonat...  ...  soc.religion.christian
3  [From, :, dlecoint, @, garnet.acns.fsu.edu, (,...  ...  soc.religion.christian
4  [From, :, a137490, @, lehtori.cc.tut.fi, (, Aa...  ...             alt.atheism

[5 rows x 3 columns]
['From', ':', 'mathew', '<', 'mathew', '@', 'mantis.co.uk', '>', 'Subject', ':', 'Re', ':', '<', 'Political', 'Atheists', '?', 'Organization', ':', 'Mantis', 'Consultants', ',', 'Cambridge.', 'UK.', 'X-Newsreader', ':', 'rusnews', 'v1.01', 'Lines', ':', '22', 'kmr4', '@', 'po.CWRU.edu', '(', 'Keith', 'M.', 'Ryan', ')', 'writes', ':', '>', '(', 'I', 'am', 'almost', 'sure', 'that', 'Zyklon-B', 'is', 'immediate', 'and', 'painless', 'method', 'of', '>', 'death.', 'If', 'not', ',', 'ins

In [15]:
# Stop words removal
def remove_stop_words(dataset):
    stop_words = set(stopwords.words('english'))
    for i in range(dataset.shape[0]):
        dataset["article"][i] = ([token.lower() for token in dataset["article"][i] if token not in stop_words])
    
    return dataset

In [16]:
df = remove_stop_words(df)
print(df.head())
print(df["article"][0])

                                             article  ...              taget_name
0  [from, :, mathew, <, mathew, @, mantis.co.uk, ...  ...             alt.atheism
1  [from, :, keith, @, cco.caltech.edu, (, keith,...  ...             alt.atheism
2  [from, :, jonh, @, david.wheaton.edu, (, jonat...  ...  soc.religion.christian
3  [from, :, dlecoint, @, garnet.acns.fsu.edu, (,...  ...  soc.religion.christian
4  [from, :, a137490, @, lehtori.cc.tut.fi, (, aa...  ...             alt.atheism

[5 rows x 3 columns]
['from', ':', 'mathew', '<', 'mathew', '@', 'mantis.co.uk', '>', 'subject', ':', 're', ':', '<', 'political', 'atheists', '?', 'organization', ':', 'mantis', 'consultants', ',', 'cambridge.', 'uk.', 'x-newsreader', ':', 'rusnews', 'v1.01', 'lines', ':', '22', 'kmr4', '@', 'po.cwru.edu', '(', 'keith', 'm.', 'ryan', ')', 'writes', ':', '>', '(', 'i', 'almost', 'sure', 'zyklon-b', 'immediate', 'painless', 'method', '>', 'death.', 'if', ',', 'insert', 'soem', 'form.', ')', '>', '>', 'a

In [17]:
# Normalization
def normalize(dataset):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    for i in range(dataset.shape[0]):
        dataset["article"][i] = " ".join([lemmatizer.lemmatize(token) for token in dataset["article"][i]]).strip()
    
    return dataset

In [18]:
df = normalize(df)
print(df.head())
print(df["article"][0])

                                             article  ...              taget_name
0  from : mathew < mathew @ mantis.co.uk > subjec...  ...             alt.atheism
1  from : keith @ cco.caltech.edu ( keith allan s...  ...             alt.atheism
2  from : jonh @ david.wheaton.edu ( jonathan hay...  ...  soc.religion.christian
3  from : dlecoint @ garnet.acns.fsu.edu ( darius...  ...  soc.religion.christian
4  from : a137490 @ lehtori.cc.tut.fi ( aario sam...  ...             alt.atheism

[5 rows x 3 columns]
from : mathew < mathew @ mantis.co.uk > subject : re : < political atheist ? organization : mantis consultant , cambridge. uk. x-newsreader : rusnews v1.01 line : 22 kmr4 @ po.cwru.edu ( keith m. ryan ) writes : > ( i almost sure zyklon-b immediate painless method > death. if , insert soem form. ) > > and , ethnic minority group killed , mutilated > exterminated history , i guess unusual. > > so , would agree holocost would allowed u > constitution ? [ far , punishment. i doubt rec

In [19]:
# Punctuation and Symbols removal
def remove_garbage(dataset):
    garbage = "~`!@#$%^&*()_-+={[}]|\:;'<,>.?/"
    for i in range(dataset.shape[0]):
        dataset["article"][i] = "".join([char for char in dataset["article"][i] if char not in garbage])
    return dataset

In [20]:
df = remove_garbage(df)
print(df.head())
print(df["article"][0])

                                             article  ...              taget_name
0  from  mathew  mathew  mantiscouk  subject  re ...  ...             alt.atheism
1  from  keith  ccocaltechedu  keith allan schnei...  ...             alt.atheism
2  from  jonh  davidwheatonedu  jonathan hayward ...  ...  soc.religion.christian
3  from  dlecoint  garnetacnsfsuedu  dariuslecoin...  ...  soc.religion.christian
4  from  a137490  lehtoricctutfi  aario sami  sub...  ...             alt.atheism

[5 rows x 3 columns]
from  mathew  mathew  mantiscouk  subject  re   political atheist  organization  mantis consultant  cambridge uk xnewsreader  rusnews v101 line  22 kmr4  pocwruedu  keith m ryan  writes    i almost sure zyklonb immediate painless method  death if  insert soem form    and  ethnic minority group killed  mutilated  exterminated history  i guess unusual   so  would agree holocost would allowed u  constitution   far  punishment i doubt recieved would  considered  fair  trial u standards

# Feature Extraction

In [21]:
def fit_corpus(train_data):
    tfidf = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1,6))
    tfidf.fit(train_data["article"])
    return tfidf

In [22]:
def transform_data(tfidf, dataset):
    features = tfidf.transform(dataset["article"])
    return pd.DataFrame(features.todense(), columns = tfidf.get_feature_names())

In [23]:
tfidf = fit_corpus(df)  #Fitting the vectorizer
train_features = transform_data(tfidf, df)  #transforming 
train_labels = df["target_code"]  #Taking lables in separate

In [24]:
train_features.head()

,00,00 these,00 these opinion,00 these opinion probably,00 these opinion probably show,00 these opinion probably show know,0001,0001 south,0001 south africa,0001 south africa internet,0001 south africa internet hayesstw,0001 south africa internet hayesstw risc1unisaacza,0100,01580,01580 change,01580 change without,01580 change without notice,01580 change without notice when,01580 change without notice when dream,02,02173,02173 are,02173 are 6179812575,02173 are 6179812575 cs,02173 are 6179812575 cs lewis,04,04 tsakc,04 tsakc my,04 tsakc my thought,04 tsakc my thought post,04 tsakc my thought post idea,0500,0500 from,0500 from nanci,0500 from nanci ann,0500 from nanci ann miller,0500 from nanci ann miller nm0w,051018,051018 gmt,051018 gmt bobbe,...,yoyoccmonasheduau fred rice wrote,yugoslavia,zaphodmpsohiostateedu,zaphodmpsohiostateedu wupost,zaphodmpsohiostateedu wupost uunet,zaphodmpsohiostateedu wupost uunet olivea,zaphodmpsohiostateedu wupost uunet olivea sgigate,zaphodmpsohiostateedu wupost uunet olivea sgigate odin,zazen,zazen austinibmcom,zazen austinibmcom welbon,zazen austinibmcom welbon there,zazen austinibmcom welbon there mean,zazen austinibmcom welbon there mean possibly,zealand,zealand line,zebedee,zebedee certainly,zebedee certainly rome,zebedee certainly rome hand,zebedee certainly rome hand first,zebedee certainly rome hand first martyr,zechariah,zen,zen buddhist,zeuscalpolyedu,zeuscalpolyedu jmunch,zeuscalpolyedu jmunch hertzeleecalpolyedu,zeuscalpolyedu jmunch hertzeleecalpolyedu john,zeuscalpolyedu jmunch hertzeleecalpolyedu john munch,zeuscalpolyedu jmunch hertzeleecalpolyedu john munch wrote,zillion,zlumber,zoerasterism,zoerasterism shintoism,zoerasterism shintoism islam,zoerasterism shintoism islam fit,zoerasterism shintoism islam fit bit,zoerasterism shintoism islam fit bit logic,zwart
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
train_labels.head()

0    0
1    0
2    1
3    1
4    0
Name: target_code, dtype: int64

In [26]:
X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.30, random_state=23)

In [27]:
print(train_features.shape)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(1079, 128016)
(755, 128016) (755,) (324, 128016) (324,)


# Model Training

## Logistic Regression

In [28]:
from sklearn.linear_model import LogisticRegression
logisticClassifier = LogisticRegression(random_state=0, solver='lbfgs')
logisticClassifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [29]:
predictions = logisticClassifier.predict(X_test)

In [30]:
print(predictions[:10])
print(y_test.values[:10])

[1 1 1 1 1 0 1 0 1 1]
[1 1 1 1 1 0 1 0 1 1]


In [31]:
np.mean(predictions == y_test)

0.941358024691358

## Naive Bayes

In [32]:
from sklearn.naive_bayes import MultinomialNB
multinomialClassifier = MultinomialNB()
multinomialClassifier.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [33]:
predictions = multinomialClassifier.predict(X_test)

In [34]:
print(predictions[:10])
print(y_test.values[:10])

[1 1 1 1 1 0 1 0 1 1]
[1 1 1 1 1 0 1 0 1 1]


In [35]:
np.mean(predictions == y_test)

0.9753086419753086

## Support vector machines - SGD

In [36]:
# Training Support Vector Machines - SVM and calculating its performance

from sklearn.linear_model import SGDClassifier
svmClassifier = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=5, random_state=42)
svmClassifier.fit(X_train, y_train)

SGDClassifier(alpha=0.001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=5,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=42, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [37]:
predictions = svmClassifier.predict(X_test)

In [38]:
print(predictions[:10])
print(y_test.values[:10])

[1 1 1 1 1 0 1 0 1 1]
[1 1 1 1 1 0 1 0 1 1]


In [39]:
np.mean(predictions == y_test)

0.9660493827160493

## Support vector machines - Linear SVM

In [ ]:
from sklearn.svm import SVC
linearSvmClassifier = SVC(kernel="linear", C=0.025)
linearSvmClassifier.fit(X_train, y_train)

In [ ]:
predictions = linearSvmClassifier.predict(X_test)

In [ ]:
print(predictions[:10])
print(y_test.values[:10])

In [ ]:
np.mean(predictions == y_test)

## Support vector machines - RBF

In [ ]:
from sklearn.svm import SVC
rbfSvmClassifier = SVC(gamma=2, C=1)
rbfSvmClassifier.fit(X_train, y_train)

In [ ]:
predictions = rbfSvmClassifier.predict(X_test)

In [ ]:
print(predictions[:10])
print(y_test.values[:10])

In [ ]:
np.mean(predictions == y_test)

## Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knnClassifier = KNeighborsClassifier()
knnClassifier.fit(X_train, y_train)

In [ ]:
predictions = knnClassifier.predict(X_test)

In [ ]:
print(predictions[:10])
print(y_test.values[:10])

In [ ]:
np.mean(predictions == y_test)

## Gaussian Process

In [ ]:
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier

GPClassifier = GaussianProcessClassifier(1.0 * RBF(1.0))
GPClassifier.fit(X_train, y_train)

In [ ]:
predictions = GPClassifier.predict(X_test)

In [ ]:
print(predictions[:10])
print(y_test.values[:10])

In [ ]:
np.mean(predictions == y_test)

## AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
adaClassifier = AdaBoostClassifier()
adaClassifier.fit(X_train, y_train)

In [ ]:
predictions = adaClassifier.predict(X_test)

In [ ]:
print(predictions[:10])
print(y_test.values[:10])

In [ ]:
np.mean(predictions == y_test)

## QDA (QuadraticDiscriminantAnalysis)

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
quadClassifier = QuadraticDiscriminantAnalysis()
quadClassifier.fit(X_train, y_train)

In [ ]:
predictions = quadClassifier.predict(X_test)

In [ ]:
print(predictions[:10])
print(y_test.values[:10])

In [ ]:
np.mean(predictions == y_test)

## Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier
decisionClassifier = DecisionTreeClassifier(random_state=23)
decisionClassifier.fit(X_train, y_train) 

In [ ]:
predictions = decisionClassifier.predict(X_test)

In [ ]:
print(predictions[:10])
print(y_test.values[:10])

In [ ]:
np.mean(predictions == y_test)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfClassifier = RandomForestClassifier(random_state=23)
rfClassifier.fit(X_train, y_train)

In [ ]:
predictions = rfClassifier.predict(X_test)

In [ ]:
print(predictions[:10])
print(y_test.values[:10])

In [ ]:
np.mean(predictions == y_test)

## Neural Net (sklearn)

In [ ]:
from sklearn.neural_network import MLPClassifier
nnClassifier = MLPClassifier(alpha=1, max_iter=1000)
nnClassifier.fit(X_train, y_train)

In [ ]:
predictions = nnClassifier.predict(X_test)

In [ ]:
print(predictions[:10])
print(y_test.values[:10])

In [ ]:
np.mean(predictions == y_test)

## XGboost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
D_train = xgb.DMatrix(X_train, label=y_train)
D_test = xgb.DMatrix(X_test, label=y_test)

In [ ]:
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 3} 

steps = 100  # The number of training iterations

In [ ]:
xgb = XGBClassifier(n_estimators=100)
xgb.fit(X_train, y_train)

In [ ]:
predictions = xgb.predict(X_test)

In [ ]:
print(predictions[:10])
print(y_test.values[:10])

In [ ]:
np.mean(predictions == y_test)